In [6]:
import torch
from torchvision.models import resnet101, ResNet

b, k, c, h, w = 4, 8, 1024, 14, 14

In [2]:
a = torch.randn(b, k, h, w)
b = torch.randn(b, c, h, w)

In [5]:
res1 = torch.einsum('bkhw,bchw->bkchw', a, b).mean((-1,-2)).permute(0,2,1)
res2 = torch.einsum('bkhw,bchw->bckhw', a, b).mean((-1,-2))

torch.equal(res1, res2)

True

In [9]:
resnet101(weights=None).fc.in_features

2048

In [11]:
torch.randn(16,200).argmax(-1).shape

torch.Size([16])